# 作者：Evan_wu        
# Vx号：Evanwu50020
#  学校：NUIST
## $DQN$算法

$DQN$算法的提出是因为当$Q-learning$算法遇到状态过大的环境时会出现维度爆炸的问题,因为$Q-learning$算法是用一个表格存储了所有状态的价值函数，所以当状态很大时，这个表格就会变得非常大，并且每次更新这个表格也会变得比较困难。

正是在这样的背景之下提出了$DQN$算法，$DQN$算法不再用表格的形式存储价值函数，而是采用神经网络来近似估计价值函数，当我们输入一个状态时，神经网络会自动近似估计出当前状态的价值。所以其实$DQN$是一个基于价值的算法，并且最主要思想就是用神经网络替换掉了$Q-table$(也就是刚才我们说的那个表格)

我们刚才提到，$DQN$算法中，我们用神经网络近似估计价值函数，但在强化学习中，我们一共有两种价值函数，所以接下来我们就会对这两种价值函数分别进行介绍。

$1.V_\pi(s):$状态价值函数

状态价值函数是指，假设现在我们的$agent$的策略是$\pi$,那么我们用该策略$\pi$和环境交互，假如现在策略$\pi$看到了一个状态$s$，我们的状态价值函数就会根据目前看到的$s$输出一个标量，这个标量就表示智能体用策略$\pi$去玩游戏，当它看到了状态$s$时，预期到游戏结束时能获得多少的$reward$。$\bf{这里需要强调的是，V_\pi(s)并不能凭空对一个state进行估计其状态价值，它所评价的是给定某一个状态的时候，如果接下来交互的agent采取的policy是\pi，那么一直按照这个\pi玩到游戏结束会获得多大的奖励，这个奖励就是我们评价得出的值。因为就算是同样的state,如果\pi不一样的话，接下来的得到的奖励也不会是一样的}$

因为我们的状态价值函数是用神经网络估算出来的，所以它一定是不太准的，那么我们应该怎么样去训练我们的神经网络，让它每次根据状态估计出来的值越来越接近真实的值呢？这里有两种方法：

①.基于蒙特卡罗采样法

在蒙特卡罗采样法中，当我们的智能体看到$s_a$时，我们记录下从这个状态到游戏结束时所获得的所有奖励，我们记为$G_a$,当智能体看到$s_b$时，我们记录下从这个状态到游戏结束时所获得的所有奖励，我们记为$G_b$.....按这样的操作，我们记录下所有的状态的真实奖励$G_a$，然后我们在到达每个状态时再用神经网络对当前状态的价值进行估计，此时我们要做的就是让神经网络在每个状态估计出来的价值函数和真实的奖励越接近越好，所以此时强化学习也就从自监督学习转换成了有标签的监督学习。

这样的方法听起来非常合理，而且按理来说应该可以训练出一个比较好的网络，但是实际操作起来这种方法很难应用，因为在蒙特卡罗采样中，我们想得到真实奖励，那我们就必然需要把这一整局玩完，但是很多游戏需要相当长的时间才能结束一次，也就是说我们会在采样上花费相当长的时间。比如在无人驾驶中，我们可能几个小时才能到达目的地，也就是说我们花几个小时才能采样到一组数据，这样的代价是无法承受的。所以这也就引出了我们的第二种矫正神经网络预测值的方法，时序差分法

②.$TD--$(时序差分法)

在时序差分法中，我们不需要像蒙特卡罗法那样玩完一整局游戏，只需要一个状态$s_t$,和在这个状态采取的动作$a_t$和采取该动作之后得到的回报$r_t$，并且进入的下一个状态$s_{t+1}$,在我们收集了多组这样的四个元素之后，我们就可以依据下面的公式开始更新我们的神经网络：$$V_\pi(s_t) = r_t + V_\pi(s_{t+1})$$我们发现我们式子两边的状态价值其实都是由我们的神经网络估计出来的，也就是都是不准的，但是在两个状态价值中相差了一个$t$时间的奖励，所以如果我们神经网络估计出来的两个相邻状态的差值越接近他们之间相差的那个奖励的值，那我们就认为我们的神经网络已经可以比较好的估计每个状态的状态价值函数。

那么上面的这两种用来矫正神经网络的方法各有什么优缺点呢？在蒙特卡罗法中，我们是在看到一个状态$s$之后用这个状态之后的整个个回合的奖励$G$来作为我们学习的标签，可是这样的问题就在于这个$G$的方差会很大，因为我们在遇到同一个之后状态之后可能在最后会得到完全不一样的两个回报$G$。所以我们可以把$G$看作是一个随机变量，而这个随机变量具有比较大的方差。同样的，我们也可以把$r$也看作是一个随机变量，因为我们在玩游戏的时候在同一个状态采取同样的动作也有可能得到不同的奖励。但是和$G$相比，$r$的方差就会小很多，因为$r$只是两个状态之间的一个奖励而已。所以从方差的角度来看，我们的时序差分法会更占优势，但是正如上面所说，我们的时序差分法的两个状态价值函数都是由神经网络估计而来，也就是当两个相邻的状态估计的很不准时，就比较难训练得到一个好的结果。而蒙特卡罗采样则是把环境真实反馈的奖励当作标签来训练神经网络，所以蒙特卡罗方法属于无偏估计，时序差分法属于有偏估计。

$2.Q_\pi(s,a):$动作状态价值函数

和状态价值函数不同的是，动作价值函数的输入不再是单纯的一个状态，而是一个状态-动作对，其指在某一个状态采取某一个动作，假设我们都使用策略$\pi$，得到的累计的期望奖励有多大。$Q$函数有一个需要注意的问题就是，策略$\pi$在看到状态$s$时不一定会采取动作$a$,这并不重要。在状态$s$强制采取动作$a$。并且接下来都用策略$\pi$一直玩下去，得到的期望奖励会是$Q_\pi(s,a)$。

$DQN$算法是基于价值的强化学习算法，也就是说我们并没有显示地去更新我们的价值，我们是用取最大值的方式来隐性地决定我们的策略：$$\pi^{\prime}(s)=\arg\max\limits_{a}Q_\pi(s,a)$$有了$Q$函数之后，我们把通过$Q$函数决定的策略定义为$\pi^{\prime}$。$DQN$这个算法只适用于离散的动作，而$DQN$只适用于离散动作的原因就在于我们的$\arg\max\limits_{a}$操作，因为如果我们的动作是连续的值，是取不了最大值的。所以如果我们现在把直接取最大值的操作就作为新的策略，我们最后到游戏结束的总奖励一定会大于等于按照原来的策略玩游戏获得的总奖励。也就是说，我们虽然并没有学习一个专门的网络用来决定策略，但是我们可以直接根据$Q$函数的值隐性地决定我们接下来的策略。

接下来我们就要介绍$DQN$算法中用到的几个技巧：

1.目标$Q$网络的技巧

如果说我们现在收集到一组数据，比如在状态$s_t$采取动作$a_t$之后，得到奖励$r_t$，进入状态$s_{t+1}$。那么根据$Q$函数，我们得到如下式子$$Q_\pi(s_t,a_t)=r_t+Q_\pi(s_{t+1},a_{t+1})$$也就是说我们希望我们给神经网络输入数据的时候，当我们输入$(s_t,a_t)$和$(s_{t+1},a_{t+1})$时，这一组相邻的两个数组之间的差值可以非常接近$r_t$,这与时序差分法的思想是一致的。但是我们会发现如果只是这样去训练网络的话往往得不到非常好的结果，因为我们的输入和目标是由同一个网络估计得到的，也就是说当我们更新一次我们的网络权重之后，我们输入同样的状态，我们的结果会变得不一样，这意味着我们的目标是一直在变动的，这会让我们的训练过程变得非常奇怪。

所以我们在训练的时候，通常会把其中一个$Q$网络固定住，在训练时，我们只会对左边$Q$网络的参数进行更新，而右边$Q$网络的参数会被固定住。因为右边网络用来计算我们目标，所以我们也把右边的网络称为目标网络。当目标网络固定住之后，我们得到的目标也就被固定了，所以此时这又转换成了一个回归问题，我们只要让我们左边的网络的输出值和我们右边网络的目标值越接近越好。在实际操作中，我们往往会先把左边的网络更新多次之后，再把左边网络的参数都复制给目标网络。

2.探索的技巧

当我们使用$Q$函数来决定我们的策略时，我们会发现这其实并不是一个好的策略，因为我们此时只会采取$Q$值最大的那个动作。这就可能会出现一个问题，比如说我现在处在某一个状态$s$,并且我现在有三个动作可以选择，其中选择$a_2$会令我此时的动作状态价值达到最大，但是这并不是说其它动作就一定比$a_2$逊啊。也有可能$a_1$和$a_3$在当前状态的状态价值并不高，但是当我在这个状态选了这两个动作之后我在后续的游戏中可以得到更高的奖励也说不定啊。

所以说我们应该在$DQN$中加一些探索的技巧，这样有利于我们找到更好的解法。那么在$DQN$中最常用的方法就是$\epsilon-$贪心，我们会设定一个很小的值，我们将这个很小的值称为$\epsilon$,我们在选动作的时候会有一个随机概率的判断，如果说我的概率小于这个值，那么我就会随机采取一个动作，如果大于这个值，则会直接进行$\arg\max$操作。

3.经验池的技巧

经验池会构造一个回放缓冲区，回放缓冲区又被称为回放内存。回放缓冲区是指现在有一个策略$\pi$去与环境交互，它会去收集数据，我们把所有的数据都放到数据缓冲区$(buffer)$里面，数据缓冲区里存储了很多数据。这其中的每一组数据都记录了这样的一件事，我们之前在某一个状态$s_t$,采取某一个动作$a_t$,得到了奖励$r_t$,进入状态$s_{t+1}$。我们用$\pi$去与环境交互多次，把收集到的数据都放到回放缓冲区里。

采取经验池的技巧会给我们带来两个好处：

①：使用了经验池之后我们的$DQN$就变成了一个异策略的算法，因为我们的经验池中并不完全是用同一个策略采样到的数据装满的，有很多经验是其它策略所采样到的。这种异策略的算法大大提高了我们对于经验的利用率，并且提高了采样的效率。

②：我们刚才也提到了，我们的经验池中会存放来自很多策略的经验，这其实是有利于训练我们的网络的，因为如果说所有数据都是同性质的，是很难训练得到一个比较好的结果的。

In [3]:
import gym
import torch
import torch.nn as nn
import torch.optim as  optim
import numpy as np

In [3]:
class Qnet(nn.Module):
    def __init__(self,observation_dim,action_dim,hidden_size):
        super(Qnet,self).__init__()
        self.linear1 = nn.Linear(observation_dim, hidden_size)
        self.linear2 = nn.Linear(hidden_size, action_dim)
    
    def forward(self, observation):
        observation = torch.from_numpy(observation)
        out = self.linear1(observation)
        out = self.linear2(out)
        
        return out

In [4]:
import random
class Replay_buffer():
    def __init__(self,capacity):
        self.capacity = capacity
        self.state = [];self.action = [];self.next_state = [];self.reward = [];self.done = []
    
    def push(self, trajectory):
        for experience in trajectory:
            self.state.append(experience[0])
            self.action.append(experience[1])
            self.next_state.append(experience[2])
            self.reward.append(experience[3])
            self.done.append(experienc[4])
    
        while(len(self.state)> self.capacity):
            state.pop[0];action.pop[0];next_state.pop[0];reward.pop[0];done.pop[0]
        
    def __len__(self):
        return len(self.state)
    
    def sample(self,batch_size):
        indices = random.sample(range(len(state)),batch_size)
        return torch.cat(state[i] for i in indices),\
               torch.cat(action[i] for i in indices),\
               torch.cat(next_state[i] for i in indices),\
               torch.cat(reward[i] for i in indices),\
               torch.cat(done[i] for i in indices)
        
        

In [5]:
class dqn():
    def __init__(self,observation_dim,action_dim,hidden_size,device):
        self.gamma = 0.99
        self.policy = Qnet(observation_dim,action_dim,hidden_size).to(device)
        self.target_policy = Qnet(observation_dim,action_dim,hidden_size).to(device)
        self.target_policy.load_state_dict(self.policy.state_dict())
        self.optim = optim.Adam(self.policy.parameters(), lr = 0.003)
        self.loss = nn.MSELoss()
        
    def update_parameters(self, state_batch, action_batch, next_state_batch, reward_batch, done_batch,device):
        state_batch = state_batch.to(device)
        action_batch = action_batch.to(device)
        next_state_batch = next_state_batch.to(device)
        reward_batch = reward_batch.to(device)
        done_batch = done_batch.to(device)
        
        q_eval = self.policy(state_batch).gather(1,action_batch)
        q_next = self.target_policy(next_state_batch).detch()
        q_target = reward_batch + gamma * q_next.max(1)[0]
        print(q_target.shape)
        loss = self.loss(q_eval, q_target)
        
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
        

In [2]:
hidden_size = 256
env_name = "LunarLander-v2"
env = gym.make(env_name)
state = env.reset()
state_dim = env.observation_space.shape[0]
state_dim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def is_discrete(env):
    try:
        k = env.action_space.n
        return 1
    except:
        return 0
flag = is_discrete(env)
if flag:
    action_dim = env.action_space.n
    print(f"Discrete Environment with {action_dim} action dims.")
else:
    print(f"Continuous Environment.")

Discrete Environment with 4 action dims.


In [ ]:

import random
qnet = dqn(state_dim, action_dim, hidden_size,device)
iteration = 1000
e = 0.1
for i in range(iter):
    state = env.reset()
    done = False
    buffer = Replay_buffer(2000)
    trajectory = []
    total_reward = []
    episode_reward = 0
    while not done:
        if random.random<e:
            action = env.action_space.sample()
        else:
            action = int(qnet.policy(state).max(0)[1])#采取e贪婪法选择动作
        next_state,reward,done,_ = env.step(action)
        episode_reward += reward
        trajectory.append((state, action,next_state, reward, done))
        state = next_state#将下一个状态作为当前状态
        
        if buffer.length>=1000:
            qnet.update_parameters()
            
        if done:
            print(f"episode:{i},episode_reward:{episode_reward}")
            break
    total_reard.append(episode_reward)
    ax.set_slim(0,1000)
    ax.plot(total_reward,'g-',label = "reward")
    plt.pause(0.001)